# Convert TFLite model to PyTorch

This uses the model **face_detection_front.tflite** from [MediaPipe](https://github.com/google/mediapipe/tree/master/mediapipe/models).

Prerequisites:

1) Clone the MediaPipe repo:

```
git clone https://github.com/google/mediapipe.git
```

2) Install **flatbuffers**:

```
git clone https://github.com/google/flatbuffers.git
cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release
make -j

cd flatbuffers/python
python setup.py install
```

3) Clone the TensorFlow repo. We only need this to get the FlatBuffers schema files (I guess you could just download [schema.fbs](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/lite/schema/schema.fbs)).

```
git clone https://github.com/tensorflow/tensorflow.git
```

4) Convert the schema files to Python files using **flatc**:

```
./flatbuffers/flatc --python tensorflow/tensorflow/lite/schema/schema.fbs
```

Now we can use the Python FlatBuffer API to read the TFLite file!

In [1]:
# !git clone https://github.com/google/mediapipe.git
# !git clone https://github.com/google/flatbuffers.git
# !cd flatbuffers ; cmake -G "Unix Makefiles" -DCMAKE_BUILD_TYPE=Release ; make -j
# !cd flatbuffers/python ; python setup.py install
# !git clone https://github.com/tensorflow/tensorflow.git
# !./flatbuffers/flatc --python tensorflow/tensorflow/lite/schema/schema.fbs

Now restart this notebook

In [2]:
import os
import numpy as np
from collections import OrderedDict

## Get the weights from the TFLite file

Load the TFLite model using the FlatBuffers library:

In [3]:
from tflite import Model

# taken from arcore pod
data = open("./mediapipe/mediapipe/models/facemesh-lite.f16.tflite", "rb").read()
model = Model.Model.GetRootAsModel(data, 0)

In [4]:
subgraph = model.Subgraphs(0)
subgraph.Name()

b'facemesh-lite.tflite.no_meta'

In [5]:
def get_shape(tensor):
    return [tensor.Shape(i) for i in range(tensor.ShapeLength())]

List all the tensors in the graph:

In [6]:
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    print("%3d %30s %d %2d %s" % (i, tensor.Name(), tensor.Type(), tensor.Buffer(), 
                                  get_shape(subgraph.Tensors(i))))

  0                     b'input_1' 0  0 [1, 192, 192, 3]
  1             b'conv2d_1/Kernel' 1  1 [16, 3, 3, 3]
  2               b'conv2d_1/Bias' 1  2 [16]
  3                    b'conv2d_1' 0  0 [1, 96, 96, 16]
  4             b'p_re_lu_1/Alpha' 1  3 [1, 1, 16]
  5                   b'p_re_lu_1' 0  0 [1, 96, 96, 16]
  6   b'depthwise_conv2d_1/Kernel' 1  4 [1, 3, 3, 16]
  7     b'depthwise_conv2d_1/Bias' 1  5 [16]
  8          b'depthwise_conv2d_1' 0  0 [1, 96, 96, 16]
  9             b'conv2d_2/Kernel' 1  6 [16, 1, 1, 16]
 10               b'conv2d_2/Bias' 1  7 [16]
 11                    b'conv2d_2' 0  0 [1, 96, 96, 16]
 12                       b'add_1' 0  0 [1, 96, 96, 16]
 13             b'p_re_lu_2/Alpha' 1  8 [1, 1, 16]
 14                   b'p_re_lu_2' 0  0 [1, 96, 96, 16]
 15   b'depthwise_conv2d_2/Kernel' 1  9 [1, 3, 3, 16]
 16     b'depthwise_conv2d_2/Bias' 1 10 [16]
 17          b'depthwise_conv2d_2' 0  0 [1, 96, 96, 16]
 18             b'conv2d_3/Kernel' 1 11 [16, 1, 1, 1

Make a look-up table that lets us get the tensor index based on the tensor name:

In [7]:
tensor_dict = {(subgraph.Tensors(i).Name().decode("utf8")): i 
               for i in range(subgraph.TensorsLength())}

Grab only the tensors that represent weights and biases.

In [8]:
parameters = {}
for i in range(subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0:
        name = tensor.Name().decode("utf8")
        parameters[name] = tensor.Buffer()

len(parameters)

116

The buffers are simply arrays of bytes. As the docs say,

> The data_buffer itself is an opaque container, with the assumption that the
> target device is little-endian. In addition, all builtin operators assume
> the memory is ordered such that if `shape` is [4, 3, 2], then index
> [i, j, k] maps to `data_buffer[i*3*2 + j*2 + k]`.

For weights and biases, we need to interpret every 4 bytes as being as float. On my machine, the native byte ordering is already little-endian so we don't need to do anything special for that.

In [9]:
def get_weights(tensor_name):
    i = tensor_dict[tensor_name]
    tensor = subgraph.Tensors(i)
    buffer = tensor.Buffer()
    shape = get_shape(tensor)
    assert(tensor.Type() == 1)  # FLOAT16
    
    W = model.Buffers(buffer).DataAsNumpy()
    W = W.view(dtype=np.float16)
    W = W.reshape(shape)
    return W

In [10]:
W = get_weights("conv2d_1/Kernel")
b = get_weights("conv2d_1/Bias")
W.shape, b.shape

((16, 3, 3, 3), (16,))

Now we can get the weights for all the layers and copy them into our PyTorch model.

## Convert the weights to PyTorch format

In [11]:
import torch
import torch.nn as nn
from facemesh import FaceMesh

In [12]:
net = FaceMesh()

In [13]:
net

FaceMesh(
  (backbone): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2))
    (1): PReLU(num_parameters=16)
    (2): FaceMeshBlock(
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=16)
    )
    (3): FaceMeshBlock(
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=16)
        (1): Conv2d(16, 16, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=16)
    )
    (4): FaceMeshBlock(
      (max_pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (convs): Sequential(
        (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(2, 2), groups=16)
        (1): Conv2d(16, 32, kernel_size=(1, 1), stride=(1, 1))
      )
      (act): PReLU(num_parameters=32)
    )
    (5): FaceMeshBlock(
   

In [14]:
net(torch.randn(2,3,192,192))[0].shape

torch.Size([2, 3, 193, 193])


torch.Size([2, 1404])

Make a lookup table that maps the layer names between the two models. We're going to assume here that the tensors will be in the same order in both models. If not, we should get an error because shapes don't match.

In [15]:
probable_names = []
for i in range(0, subgraph.TensorsLength()):
    tensor = subgraph.Tensors(i)
    if tensor.Buffer() > 0 and tensor.Type() == 1:
        probable_names.append(tensor.Name().decode("utf-8"))
        
probable_names[:5]

['conv2d_1/Kernel',
 'conv2d_1/Bias',
 'p_re_lu_1/Alpha',
 'depthwise_conv2d_1/Kernel',
 'depthwise_conv2d_1/Bias']

In [16]:
len(probable_names)

113

In [17]:
from pprint import pprint

In [18]:
pprint(list(zip(probable_names, net.state_dict())))

[('conv2d_1/Kernel', 'backbone.0.weight'),
 ('conv2d_1/Bias', 'backbone.0.bias'),
 ('p_re_lu_1/Alpha', 'backbone.1.weight'),
 ('depthwise_conv2d_1/Kernel', 'backbone.2.convs.0.weight'),
 ('depthwise_conv2d_1/Bias', 'backbone.2.convs.0.bias'),
 ('conv2d_2/Kernel', 'backbone.2.convs.1.weight'),
 ('conv2d_2/Bias', 'backbone.2.convs.1.bias'),
 ('p_re_lu_2/Alpha', 'backbone.2.act.weight'),
 ('depthwise_conv2d_2/Kernel', 'backbone.3.convs.0.weight'),
 ('depthwise_conv2d_2/Bias', 'backbone.3.convs.0.bias'),
 ('conv2d_3/Kernel', 'backbone.3.convs.1.weight'),
 ('conv2d_3/Bias', 'backbone.3.convs.1.bias'),
 ('p_re_lu_3/Alpha', 'backbone.3.act.weight'),
 ('depthwise_conv2d_3/Kernel', 'backbone.4.convs.0.weight'),
 ('depthwise_conv2d_3/Bias', 'backbone.4.convs.0.bias'),
 ('conv2d_4/Kernel', 'backbone.4.convs.1.weight'),
 ('conv2d_4/Bias', 'backbone.4.convs.1.bias'),
 ('p_re_lu_4/Alpha', 'backbone.4.act.weight'),
 ('depthwise_conv2d_4/Kernel', 'backbone.5.convs.0.weight'),
 ('depthwise_conv2d_4/Bia

In [19]:
len(net.state_dict()), len(probable_names)

(113, 113)

In [20]:
convert = {}
i = 0
for name, params in net.state_dict().items():
    if i < 83:
        convert[name] = probable_names[i]
        i += 1

In [21]:
manual_mapping = {
    'coord_head.2.convs.0.weight': 'depthwise_conv2d_17/Kernel',
    'coord_head.2.convs.0.bias': 'depthwise_conv2d_17/Bias',
    'coord_head.2.convs.1.weight': 'conv2d_18/Kernel',
    'coord_head.2.convs.1.bias': 'conv2d_18/Bias',
    'coord_head.2.act.weight': 'p_re_lu_18/Alpha',
    'coord_head.3.weight': 'conv2d_19/Kernel',
    'coord_head.3.bias': 'conv2d_19/Bias',
    'coord_head.4.weight': 'p_re_lu_19/Alpha',
    'coord_head.5.convs.0.weight': 'depthwise_conv2d_18/Kernel',
    'coord_head.5.convs.0.bias': 'depthwise_conv2d_18/Bias',
    'coord_head.5.convs.1.weight': 'conv2d_20/Kernel',
    'coord_head.5.convs.1.bias': 'conv2d_20/Bias',
    'coord_head.5.act.weight': 'p_re_lu_20/Alpha',
    'coord_head.6.weight': 'conv2d_21/Kernel',
    'coord_head.6.bias': 'conv2d_21/Bias',
    'conf_head.0.convs.0.weight': 'depthwise_conv2d_23/Kernel',
    'conf_head.0.convs.0.bias': 'depthwise_conv2d_23/Bias',
    'conf_head.0.convs.1.weight': 'conv2d_28/Kernel',
    'conf_head.0.convs.1.bias': 'conv2d_28/Bias',
    'conf_head.0.act.weight': 'p_re_lu_26/Alpha',
    'conf_head.1.weight': 'conv2d_29/Kernel',
    'conf_head.1.bias': 'conv2d_29/Bias',
    'conf_head.2.weight': 'p_re_lu_27/Alpha',
    'conf_head.3.convs.0.weight': 'depthwise_conv2d_24/Kernel',
    'conf_head.3.convs.0.bias': 'depthwise_conv2d_24/Bias',
    'conf_head.3.convs.1.weight': 'conv2d_30/Kernel',
    'conf_head.3.convs.1.bias': 'conv2d_30/Bias',
    'conf_head.3.act.weight': 'p_re_lu_28/Alpha',
    'conf_head.4.weight': 'conv2d_31/Kernel',
    'conf_head.4.bias': 'conv2d_31/Bias'
}
convert.update(manual_mapping)

Copy the weights into the layers.

Note that the ordering of the weights is different between PyTorch and TFLite, so we need to transpose them.

Convolution weights:

    TFLite:  (out_channels, kernel_height, kernel_width, in_channels)
    PyTorch: (out_channels, in_channels, kernel_height, kernel_width)

Depthwise convolution weights:

    TFLite:  (1, kernel_height, kernel_width, channels)
    PyTorch: (channels, 1, kernel_height, kernel_width)
    
PReLU:

    TFLite:  (1, 1, num_channels)
    PyTorch: (num_channels, )


In [22]:
new_state_dict = OrderedDict()

for dst, src in convert.items():
    W = get_weights(src)
    print(dst, src, W.shape, net.state_dict()[dst].shape)

    if W.ndim == 4:
        if W.shape[0] == 1 and dst != "conf_head.4.weight":
            W = W.transpose((3, 0, 1, 2))  # depthwise conv
        else:
            W = W.transpose((0, 3, 1, 2))  # regular conv
    elif W.ndim == 3:
        W = W.reshape(-1)
    
    new_state_dict[dst] = torch.from_numpy(W)

backbone.0.weight conv2d_1/Kernel (16, 3, 3, 3) torch.Size([16, 3, 3, 3])
backbone.0.bias conv2d_1/Bias (16,) torch.Size([16])
backbone.1.weight p_re_lu_1/Alpha (1, 1, 16) torch.Size([16])
backbone.2.convs.0.weight depthwise_conv2d_1/Kernel (1, 3, 3, 16) torch.Size([16, 1, 3, 3])
backbone.2.convs.0.bias depthwise_conv2d_1/Bias (16,) torch.Size([16])
backbone.2.convs.1.weight conv2d_2/Kernel (16, 1, 1, 16) torch.Size([16, 16, 1, 1])
backbone.2.convs.1.bias conv2d_2/Bias (16,) torch.Size([16])
backbone.2.act.weight p_re_lu_2/Alpha (1, 1, 16) torch.Size([16])
backbone.3.convs.0.weight depthwise_conv2d_2/Kernel (1, 3, 3, 16) torch.Size([16, 1, 3, 3])
backbone.3.convs.0.bias depthwise_conv2d_2/Bias (16,) torch.Size([16])
backbone.3.convs.1.weight conv2d_3/Kernel (16, 1, 1, 16) torch.Size([16, 16, 1, 1])
backbone.3.convs.1.bias conv2d_3/Bias (16,) torch.Size([16])
backbone.3.act.weight p_re_lu_3/Alpha (1, 1, 16) torch.Size([16])
backbone.4.convs.0.weight depthwise_conv2d_3/Kernel (1, 3, 3, 1

In [23]:
net.load_state_dict(new_state_dict, strict=True)

<All keys matched successfully>

No errors? Then the conversion was successful!

## Save the checkpoint

In [24]:
torch.save(net.state_dict(), "facemesh.pth")